In [31]:
import os 

os.getcwd()

'/home/nittaak/661'

In [2]:
import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
from transformers import PreTrainedTokenizerFast, AutoTokenizer, AutoModelForCausalLM
import torch
from cosinesimcontained import get_cosine_sim_bert
import numpy as np
from transformers import BertTokenizer, BertModel

def load_data(data_dir, labels_dir):
    data_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.txt')])
    label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.txt')])

    dataset = []

    for data_file, label_file in zip(data_files, label_files):
        with open(os.path.join(data_dir, data_file), 'r') as df, open(os.path.join(labels_dir, label_file), 'r') as lf:
            data = df.read()
            label = lf.read()

            dataset.append((data, label))  # Append as a tuple (data, label)

    return dataset

class TextDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            data = self.transform(data)

        return data, label

model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Train Data
dataset_betham_train = load_data('/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/train/Betham/OCR', '/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/train/Betham/GT/')
dataset_IAM_train = load_data('/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/train/IAM/OCR/','/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/train/IAM/GT/')

## Validation Data
dataset_betham_val = load_data('/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/validation/Betham/OCR', '/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/validation/Betham/GT/')
dataset_IAM_val = load_data('/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/validation/IAM/OCR/','/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/validation/IAM/GT/')

## Test Data
dataset_betham_test = load_data('/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/test/Betham/OCR/', '/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/test/Betham/GT/')
dataset_IAM_test = load_data('/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/test/IAM/OCR/','/Users/amandanitta/Desktop/GitHub/ICS661/ICS661-proj/digitalize_handwritten/dataForModel/test/IAM/GT/')

In [3]:
len_data_bentham = []
len_labels_bentham = []

len_data_IAM = []
len_labels_IAM = []

for data, label in dataset_betham_val:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    len_data_bentham.append(len(data))
    len_labels_bentham.append(len(label))

for data, label in dataset_IAM_val:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    len_data_IAM.append(len(data))
    len_labels_IAM.append(len(label))

In [4]:
for data, label in dataset_betham_train:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    len_data_bentham.append(len(data))
    len_labels_bentham.append(len(label))

for data, label in dataset_IAM_train:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    len_data_IAM.append(len(data))
    len_labels_IAM.append(len(label))

In [25]:
for data, label in dataset_betham_test:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    len_data_bentham.append(len(data))
    len_labels_bentham.append(len(label))

for data, label in dataset_IAM_test:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    len_data_IAM.append(len(data))
    len_labels_IAM.append(len(label))

len_overall_handwritten = len_labels_bentham + len_labels_IAM

In [30]:
# https://www.statology.org/confidence-interval-for-median/ 

"""
Lower: j = median - z*sqrt(1-q)
Upper: k = median + z*sqrt(1-q)
"""
z_value = 1.96
quartile = 0.5
total_IAM = len(len_labels_IAM)

confidence_IAM = z_value*np.sqrt(total_IAM*quartile*(1-quartile))
len_labels_IAM = sorted(len_labels_IAM)
confidence_left_IAM = len_labels_IAM[-int(np.floor(confidence_IAM))]
confidence_right_IAM = len_labels_IAM[int(np.floor(confidence_IAM))]

In [31]:
z_value = 1.96
quartile = 0.5
total_bentham = len(len_labels_bentham)

confidence_bentham = z_value*np.sqrt(total_bentham*quartile*(1-quartile))
len_labels_bentham = sorted(len_labels_bentham)
confidence_left_bentham = len_labels_bentham[-int(np.floor(confidence_bentham))]
confidence_right_bentham = len_labels_bentham[int(np.floor(confidence_bentham))]

In [33]:
z_value = 1.96
quartile = 0.5
total_overall = len(len_overall_handwritten)

confidence_overall = z_value*np.sqrt(total_overall*quartile*(1-quartile))
len_overall_handwritten = sorted(len_overall_handwritten)
confidence_left_overall = len_overall_handwritten[-int(np.floor(confidence_overall))]
confidence_right_overall = len_overall_handwritten[int(np.floor(confidence_overall))]

In [32]:
import numpy as np

print(f'IAM Dataset')
print(f'Min: {np.min(len_labels_IAM)}')
print(f'Max: {np.max(len_labels_IAM)}')
print(f'Median {np.median(len_labels_IAM)}')
print(f'Total Documents {len(len_labels_IAM)}')
print(f'Confidence: [{confidence_right_IAM},{confidence_left_IAM}]')
print("-------------------------\n")

print('Bentham Dataset')
print(f'Min: {np.min(len_labels_bentham)}')
print(f'Max: {np.max(len_labels_bentham)}')
print(f'Median: {np.median(len_labels_bentham)}')
print(f'Total Documents: {len(len_labels_bentham)}')
print(f'Confidence: [{confidence_right_bentham},{confidence_left_bentham}]')

print("-------------------------\n")
print('Overall Dataset')
print(f'Min: {np.min(len_overall_handwritten)}')
print(f'Max: {np.max(len_overall_handwritten)}')
print(f'Median: {np.median(len_overall_handwritten)}')
print(f'Total Documents: {len(len_overall_handwritten)}')
print(f'Confidence: [{confidence_left_overall}, {confidence_right_overall}]')

IAM Dataset
Min: 58
Max: 903
Median 375.0
Total Documents 1251
Confidence: [266,564]
-------------------------

Bentham Dataset
Min: 133
Max: 3452
Median: 1160.0
Total Documents: 473
Confidence: [595,1896]
-------------------------

Overall Dataset
Min: 58
Max: 3452
Median: 409.0
Total Documents: 1724
Confidence: [408, 777]


In [33]:
cosine_sim_val = []
count_val = 0
device = torch.device("cuda:0")
model.to(device)

for data, label in dataset_betham_val:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    cosine_sim_val.append( get_cosine_sim_bert(data, label, tokenizer=tokenizer, model=model,device=device))
    count_val += 1


for data, label in dataset_IAM_val:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    cosine_sim_val.append( get_cosine_sim_bert(data, label, tokenizer=tokenizer, model=model,device=device))
    count_val += 1

cosine_sim_val_avg = np.mean(cosine_sim_val[0])

In [34]:
print(f'cosine similarity validation average: {cosine_sim_val_avg}')

cosine similarity validation average: 0.7338921888396631


In [36]:
device = torch.device("cuda:0")
model.to(device)

cosine_sim_test = []
count_test = 0

for data, label in dataset_betham_test:
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    cosine_sim_test.append( get_cosine_sim_bert(data, label, tokenizer=tokenizer, model=model,device=device))

for data, label in dataset_IAM_test: 
    if data is None or label is None: 
        continue
    if len(data) == 0 or len(label) == 0: 
        continue
    cosine_sim_test.append( get_cosine_sim_bert(data, label, tokenizer=tokenizer, model=model,device=device))
    
cosine_sim_test_avg = np.mean(cosine_sim_test[0])

In [38]:
print(f'cosine similarity test average: {cosine_sim_test_avg}')

cosine similarity test average: 0.7383061489172741
